In [1]:
import time
import torch
from torch.utils.data import TensorDataset, DataLoader


import numpy as np
import sys
sys.path.append("../..") 
import kinematics.allegro_hand as allegro

%load_ext autoreload
%autoreload 2

In [2]:
from nn_model import Net
model = Net(48, 10)


In [3]:


filepath = 'models/linear4_error2.7'


model.load_state_dict(torch.load(filepath))

<All keys matched successfully>

In [4]:
model.eval()


Net(
  (l1): Linear(in_features=48, out_features=250, bias=True)
  (l2): Linear(in_features=250, out_features=130, bias=True)
  (l3): Linear(in_features=130, out_features=50, bias=True)
  (l4): Linear(in_features=50, out_features=10, bias=True)
  (act_relu): ReLU()
)

In [5]:
# load data
path = '../dataset/'
dataset_name = 'dataset'  
nums = 10000000
suffix = '_15dis_sphere_2'
# dataset_name = 'test100000.txt'  # 59min 100k nums
# data2 = np.loadtxt(path+dataset_name, delimiter=' ')  # 
data2 = np.load(path + dataset_name + str(nums) + suffix + '.npy')   #  # 16 joints, 3 obj pose, 15 min_dis
nums = int(data2.shape[0] * 0.9)
k = 4
# data2 = data2[nums*k:nums*(k+1),:]
data2 = data2[nums:,:]
print('data2 shape:', data2.shape)


dis_with_obj = list(np.array([5,9,12,14,15]) + 18)
dis_with_obj +=[16,17,18]
# dis_with_obj
dis_only_hand = []
for i in range(34):
    if i not in dis_with_obj:
        dis_only_hand.append(i)
print(dis_only_hand)
data1 = data2[:, dis_only_hand] # 16 joints, 10 min_dis
print('data1 shape:', data1.shape)


data2 shape: (1000000, 34)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 19, 20, 21, 22, 24, 25, 26, 28, 29, 31]
data1 shape: (1000000, 26)


In [6]:
hand = allegro.Robot(use_fingers=[1,1,1,1], path_prefix='../../', all_link_fk=True, meshes=True)
lb = hand.joint_lb
ub = hand.joint_ub

del hand


[1, 1, 1, 1]
finger number= 4   joint list: [['joint_0', 'joint_1', 'joint_2', 'joint_3'], ['joint_4', 'joint_5', 'joint_6', 'joint_7'], ['joint_8', 'joint_9', 'joint_10', 'joint_11'], ['joint_12', 'joint_13', 'joint_14', 'joint_15']]


In [7]:
lb

array([-0.59471317, -0.29691277, -0.27401187, -0.32753606, -0.59471317,
       -0.29691277, -0.27401187, -0.32753606, -0.59471317, -0.29691277,
       -0.27401187, -0.32753606,  0.3635739 , -0.2050429 , -0.28972295,
       -0.26220637])

In [8]:
# normalize data to [-1,1]

data = np.copy(data1)
for i in range(16):  # joint angles
    data[:,i] = (data1[:,i] - lb[i] )/ (ub[i] - lb[i]) *2 -1

# max_dis = np.max(data1[:, 16:] )
# for i in range(16,26):  # 10 dis
#     data[:,i] = data[:,i] /max_dis


add_sin_cos = True
keep_all_dis = True

num = int(data.shape[0] * 0.8)
batch_size = 128 * 64 *4


if add_sin_cos:
    num_s = 16 * 3
    if keep_all_dis:
        data = np.concatenate([data[:,:16], np.sin(data[:,:16]), np.cos(data[:,:16]), data[:,16:]], axis=1)
    else:
        min_dis = np.min(data1[:, 16:], axis=1 ).reshape(-1, 1)
        data = np.concatenate([data[:,:16], np.sin(data[:,:16]), np.cos(data[:,:16]), min_dis], axis=1)
else:
    num_s = 16
    if keep_all_dis is False:
        min_dis = np.min(data1[:, 16:], axis=1 ).reshape(-1, 1)
        data = np.concatenate([data[:,:16], min_dis], axis=1)


x_train = torch.Tensor(data[:num, :num_s])
y_train = torch.Tensor(data[:num, num_s:])
dim_in = x_train.size(1)
dim_out = y_train.size(1)
# dataset_train = TensorDataset(x_train,y_train) # create your datset
# loader_train = DataLoader(dataset_train,batch_size=batch_size, num_workers=12)
x_test = torch.Tensor(data[num:, :num_s])
y_test = torch.Tensor(data[num:, num_s:])

print(dim_in, dim_out)

48 10


In [9]:
# verification
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x_test_1 = x_test.to(device)
y_test_1 = y_test.to(device)
model.to(device)

with torch.no_grad():
    outputs = model(x_test_1)

outputs = outputs.cpu().numpy()

error = (data[num:, num_s:] - outputs) 
np.sqrt(np.mean(error*error, axis=0))  




array([0.65893986, 0.60410471, 0.63579101, 0.95336026, 0.52659965,
       1.25102377, 0.87506007, 0.85886652, 1.11932333, 1.18477319])

In [10]:
# accuracy of collision avoidance

## in testing set, for 10 distances
dis_bool = data[num:, num_s:] <= 0
dis_bool_p = outputs <=0
c = np.equal(dis_bool, dis_bool_p)
print('Test, accuracy for 10 dis',np.sum(c)/ c.shape[0] /c.shape[1] )


# only compare the minimal distance
dis_bool = np.min(data[num:, num_s:], axis=1) <= 0
dis_bool_p = np.min(outputs, axis=1)  <= 0
c = np.equal(dis_bool, dis_bool_p)
print('Test, accuracy for min dis',np.sum(c)/ len(c) )



Test, accuracy for 10 dis 0.9933529999999999
Test, accuracy for min dis 0.96593


In [11]:
q0 = np.ones(16)
q0[12] = 0.5
if len(q0.shape) == 1:
    q = q0.reshape(1, -1)
q_all = np.concatenate([q, np.sin(q), np.cos(q)], axis=1)
# transfer q to [-1,1]

for i in range(16):  # joint angles
    q_all[:, i] = (q_all[:, i] - lb[i]) / (ub[i] - lb[i]) * 2 - 1

q_all = torch.Tensor(q_all.reshape(1,-1))
q_all = q_all.to(device)
with torch.no_grad():
    outputs = model(q_all)



outputs = outputs.cpu().numpy()

In [12]:
outputs




array([[0.31125247, 0.25174358, 0.24208462, 0.9791069 , 0.56241095,
        0.9716926 , 0.43945563, 0.7842885 , 0.21170121, 0.50430954]],
      dtype=float32)